# 🔧 GSE182159 배치 파일 재통합

**목적**: 저장된 배치 파일들을 올바르게 통합하여 GSE182159_processed.h5ad 재생성

| 배치 | 세포 수 |
|------|--------|
| batch_0 | 31,233 |
| batch_1 | 36,988 |
| batch_2 | 44,922 |
| batch_3 | 80,837 |
| batch_4 | 49,020 |
| **총합** | **243,000** |

In [ ]:
# @title STEP 1: 환경 설정
from google.colab import drive
drive.mount('/content/drive')

!pip install anndata scanpy pandas -q

import anndata as ad
import pandas as pd
import os

RAW_PATH = '/content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data'
print(f'✅ 환경 설정 완료!')
print(f'📁 경로: {RAW_PATH}')

In [ ]:
# @title STEP 2: 배치 파일 확인
batch_files = sorted([f for f in os.listdir(RAW_PATH) if f.startswith('GSE182159_batch_')])
print(f'📦 발견된 배치 파일: {len(batch_files)}개\n')

total_cells = 0
for bf in batch_files:
    adata_temp = ad.read_h5ad(f'{RAW_PATH}/{bf}', backed='r')
    print(f'   {bf}: {adata_temp.n_obs:,} cells, {adata_temp.n_vars:,} genes')
    total_cells += adata_temp.n_obs

print(f'\n📊 총 세포 수: {total_cells:,}')

In [ ]:
# @title STEP 3: 배치 파일 통합
import gc

print('🔗 배치 파일 통합 중...')
print('   (이 작업은 몇 분이 소요될 수 있습니다)\n')

adata_list = []
for i, bf in enumerate(batch_files):
    print(f'   Loading {bf}...')
    adata_batch = ad.read_h5ad(f'{RAW_PATH}/{bf}')
    adata_list.append(adata_batch)
    print(f'   ✅ {bf}: {adata_batch.n_obs:,} cells loaded')
    gc.collect()

print(f'\n🔗 Concatenating {len(adata_list)} batches...')
adata = ad.concat(adata_list, join='outer')
adata.var_names_make_unique()

print(f'\n✅ 통합 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'   샘플 수: {adata.obs["sample"].nunique()}')

# 메모리 정리
del adata_list
gc.collect()

In [ ]:
# @title STEP 4: 메타데이터 매핑 (HBV 상태 추가)
metadata_file = f'{RAW_PATH}/sample_metadata.csv'

if os.path.exists(metadata_file):
    print('📋 메타데이터 로드 중...')
    metadata_df = pd.read_csv(metadata_file)
    print(f'   샘플 수: {len(metadata_df)}')
    print(f'   컬럼: {metadata_df.columns.tolist()}')
    
    # GSM ID 추출 (샘플명에서)
    # sample 형식: GSM5519467_P190604_Blood_1
    adata.obs['gsm_id'] = adata.obs['sample'].str.extract(r'(GSM\d+)')[0]
    
    # 매핑 생성
    meta_dict = metadata_df.set_index('sample_id').to_dict()
    
    # Stage 매핑 (HBV 상태)
    if 'Stage' in meta_dict:
        adata.obs['hbv_stage'] = adata.obs['gsm_id'].map(meta_dict['Stage'])
        print(f'\n📊 HBV Stage 분포:')
        print(adata.obs['hbv_stage'].value_counts())
    
    # donor 매핑
    if 'donor' in meta_dict:
        adata.obs['donor'] = adata.obs['gsm_id'].map(meta_dict['donor'])
        print(f'\n📊 Donor 수: {adata.obs["donor"].nunique()}')
    
    print('\n✅ 메타데이터 매핑 완료!')
else:
    print(f'⚠️ 메타데이터 파일이 없습니다: {metadata_file}')

In [ ]:
# @title STEP 5: 통합 파일 저장
output_file = f'{RAW_PATH}/GSE182159_processed.h5ad'

print(f'💾 저장 중: {output_file}')
print('   (이 작업은 몇 분이 소요될 수 있습니다)\n')

adata.write_h5ad(output_file)

print('=' * 60)
print('🎉 GSE182159 데이터 통합 및 저장 완료!')
print('=' * 60)
print(f'\n📁 저장된 파일: {output_file}')
print(f'\n📊 데이터 요약:')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'   샘플 수: {adata.obs["sample"].nunique()}')
print(f'\n📋 메타데이터 컬럼:')
print(f'   {list(adata.obs.columns)}')
print('\n✅ 다음 단계: Step1_HBV_real.ipynb 실행')

In [ ]:
# @title (선택) 저장된 파일 검증
print('🔍 저장된 파일 검증 중...')
adata_check = ad.read_h5ad(output_file, backed='r')
print(f'   Shape: {adata_check.shape}')
print(f'   ✅ 검증 완료! 세포 수: {adata_check.n_obs:,}')